In [12]:
import pandas as pd
from web3 import Web3
import time

# Step 1: Load CSV
df = pd.read_csv("iot_data.csv")
print("🧾 Columns in CSV:", df.columns.tolist())
print(df.head())

# Step 2: Connect to Ganache
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if web3.is_connected():
    print("✅ Connected to Ganache!")
else:
    raise Exception("❌ Could not connect to Ganache. Please check RPC URL.")

# Step 3: Contract Setup
contract_address = "0x09F6166C62739ef98F55829649971D546dEdAa2e"

abi = [
    {
        "inputs": [],
        "stateMutability": "nonpayable",
        "type": "constructor"
    },
    {
        "anonymous": False,
        "inputs": [
            {"indexed": False, "internalType": "uint256", "name": "timestamp", "type": "uint256"},
            {"indexed": False, "internalType": "string", "name": "packageId", "type": "string"},
            {"indexed": False, "internalType": "string", "name": "category", "type": "string"},
            {"indexed": False, "internalType": "string", "name": "value", "type": "string"}
        ],
        "name": "DataStored",
        "type": "event"
    },
    {
        "inputs": [],
        "name": "MAX_ENTRIES",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "name": "dataRecords",
        "outputs": [
            {"internalType": "uint256", "name": "timestamp", "type": "uint256"},
            {"internalType": "string", "name": "packageId", "type": "string"},
            {"internalType": "string", "name": "category", "type": "string"},
            {"internalType": "string", "name": "value", "type": "string"}
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [{"internalType": "uint256", "name": "index", "type": "uint256"}],
        "name": "getData",
        "outputs": [
            {"internalType": "uint256", "name": "timestamp", "type": "uint256"},
            {"internalType": "string", "name": "packageId", "type": "string"},
            {"internalType": "string", "name": "category", "type": "string"},
            {"internalType": "string", "name": "value", "type": "string"}
        ],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "getDataCount",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "owner",
        "outputs": [{"internalType": "address", "name": "", "type": "address"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {"internalType": "string", "name": "packageId", "type": "string"},
            {"internalType": "string", "name": "category", "type": "string"},
            {"internalType": "string", "name": "value", "type": "string"}
        ],
        "name": "storeData",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    }
]

contract = web3.eth.contract(address=contract_address, abi=abi)
web3.eth.default_account = web3.eth.accounts[0]
print(f"✅ Connected to Smart Contract at {contract_address}")

# Step 4: Send Data to Smart Contract
def send_iot_data(package_id, category, value):
    txn = contract.functions.storeData(package_id, category, value).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"📦 {package_id} | {category}: {value} | ✅ Txn Hash: {receipt.transactionHash.hex()}")

# Step 5: Loop through and send all data
for _, row in df.iterrows():
    package_id = str(row["packageid"])
    entries = [
        ("Temperature", f"{row['temperature_C']}°C"),
        ("Location", f"{row['location_lat']},{row['location_long']}"),
        ("Status", str(row['status'])),
        ("Tamper", str(row['tamper_detected']))
    ]
    for category, value in entries:
        send_iot_data(package_id, category, value)
        time.sleep(0.1)  # prevent flooding

# Step 6: Retrieve and display records
total_records = contract.functions.getDataCount().call()
print(f"📊 Total IoT records stored: {total_records}")

if total_records > 0:
    first_record = contract.functions.getData(0).call()
    print("📄 First Stored Record:", first_record)
else:
    print("⚠️ No records found.")



🧾 Columns in CSV: ['timestamp', 'packageid', 'product_id', 'location_lat', 'location_long', 'temperature_C', 'status', 'tamper_detected']
  timestamp packageid product_id  location_lat  location_long  temperature_C  \
0   15:21.9   PKG7795    PROD431     10.118892     120.244188            4.6   
1   00:21.9   PKG8284    PROD212     13.208275     124.515339            5.5   
2   53:21.9   PKG1259    PROD576     11.097202     121.671256            5.1   
3   23:21.9   PKG6208    PROD650     13.692627     122.950620            4.6   
4   41:21.9   PKG6492    PROD528     12.857149     124.907286            4.2   

             status  tamper_detected  
0  Out for Delivery            False  
1        In Transit            False  
2  Out for Delivery            False  
3  Out for Delivery            False  
4  Out for Delivery            False  
✅ Connected to Ganache!
✅ Connected to Smart Contract at 0x09F6166C62739ef98F55829649971D546dEdAa2e
📦 PKG7795 | Temperature: 4.6°C | ✅ Txn Hash: 55

BadFunctionCallOutput: Could not transact with/call contract function, is contract deployed correctly and chain synced?